# Case Study 2: Modeling

In [ ]:
%load_ext autoreload
%autoreload 2
import sys

sys.path.append("../python")

import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib


## Loading the dataset

We use the function described in the previous notebook:

In [ ]:
from mll1 import attrition

df_attr = attrition.load_dataset(
    "../../data/employee_attrition/Employee_attrition.xlsx"
)
df_attr

In [ ]:
len(df_attr.columns)

## Analytical dataset

We have here a pretty clean dataset. We can make the following remarks:

* there are 9 explanatory variables (the "Emp ID" is not relevant for the modeling)
* there are 5 numerical variables
* there are 2 binary variables
* there are 2 categorical variables
* the target value is binary
* there are no missing values

There does not seem to be any data cleaning needed.
This is typically what is sometimes called an **analytical dataset**, i.e. a dataset that is already prepared to be used with a machine learning algorithm.

This has the advantage to save time for the data scientist, but also the drawback that it constrains the type of analysis that can be performed on it.

Back in notebook #5, we listed some design questions we have to answer when tackling an attrition modeling task. Here, the employee status is only known at a the time the dataset was constructed.  We do not know when employees left, and do not have to deal with several time windows.  The task consists in evaluating an attrition risk for all employees who have not left yet.

Given the task and the dataset, we make the following choices:

- Approach this as a binary classification task, where the target is the variable `has_left` (1 if true, 0 if not);

- The training procedure will then aim at learning how the actual status of an employee is linked to its other observed characteristics;

- Once the model is trained, we will use the prediction for evaluating the attrition risk of all employees who have not left the company yet.  In a way, this means that our false positives (samples where the target is predicted as 1 while it is given as 0 in the dataset) are the employees at risk.  Note: we will explain in a moment the notions of true/false positive & negatives.

- Special care will be taken to avoid overfitting the dataset ; usually this is done for improving the model generalization abilities (predicting on samples that were not in the dataset at training time), but here it is vital for the model to actually learn the relationship between an employee characteristics and her/his attrition risk, and not simply "remember" the target (`has_left`).



## Performance measures

In the previous use-case we dealed with a regression task, where all performance measures were based on the prediction error. This error was measured as the difference between the predicted value and the actual value.

This would not make much sense with the classification task we have here, where by definition the target has discrete values. Instead, performance measures will be based on counts of correctly and wrongly classified data points.

### Performance measures for binary classification

In binary classification, the target can take only two values. As a convention those values are either $\{0, 1\}$, or $\{-1, 1\}$. One of the classes is usually called positive ($1$) and the other one negative ($0$ or $-1$).

When using Scikit-learn the actual encoding is not actually relevant, but this is not the case with all machine learning toolkits, and some implementation may expect a specific encoding for the target.

#### Vocabulary

When we evaluate the performance of a model, we will have a set of data points with an associated target $y_i$ (the ground truth) and a predicted value for this target $\tilde y_i$. Since there are two cases for the ground truth and two cases for the prediction we have typically 4 possible combinations:

* A **true positive** is a **positive sample** (ground truth) which is correctly **predicted as positive** ($y_i = 1$ and $\tilde y_i = 1$)
* A **false positive** is a **negative sample** (ground truth) which is wrongly **predicted as positive** ($y_i = 0$ and $\tilde y_i = 1$)
* A **true negative** is a **negative sample** (ground truth) which is correctly **predicted as negative** ($y_i = 0$ and $\tilde y_i = 0$)
* A **false negative** is a **positive sample** (ground truth) which is wrongly **predicted as negative** ($y_i = 1$ and $\tilde y_i = 0$)

The counts of all those categories is often summed up in the so-called **confusion matrix**:

<table border="1" class="dataframe">
  <thead>
    <tr>
      <th></th>
      <th></th>
      <th colspan="2" halign="left">Prediction</th>
    </tr>
    <tr>
      <th></th>
      <th></th>
      <th>0</th>
      <th>1</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th rowspan="2" valign="top">Ground Truth</th>
      <th>0</th>
      <td>TN</td>
      <td>FP</td>
    </tr>
    <tr>
      <th>1</th>
      <td>FN</td>
      <td>TP</td>
    </tr>
  </tbody>
</table>


### Performance measures

#### Accuracy

The accuracy is the rate of correctly classified samples:

$$ Acc = \frac{TP + TN}{TP + TN + FP + FN} = \frac{TP + TN}{N}$$

You might also be interested in the misclassification rate, which is the complement of the accuracy:

$$ MissRate = 1 - Acc $$

**Exercise**: What would the expected accuracy be if the "prediction" was obtained by a random permutation of the datset's target true value, and the positive / negative proportions were:

* 0.5 / 0.5
* 0.25 / 0.75
* 0.9 / 0.1

$$Acc(p) = Pr(1 \wedge 1) + Pr(0 \wedge 0) = p^2 + (1 - p)^2$$

$$\matrix{
    p = \frac{1}{2}& \rightarrow & \frac{1}{4} + \frac{1}{4} = \frac{1}{2} & = & 0.5\\
    p = \frac{1}{4}& \rightarrow & \frac{1}{16} + \frac{9}{16} = \frac{5}{8} & = & 0.625 \\
    p = \frac{9}{10}& \rightarrow & \frac{81}{100} + \frac{1}{100} = \frac{82}{100} & = & 0.82 \\
}$$

We can also estimate the expected value by simulating a random classification:

* Draw random binary values with some given probability;
* Either draw again for the predictions (with the same probability), or permutate the initial values.

Here is an example:

In [ ]:
def simulate(p, n):
    ps = [p, (1 - p)]
    values = np.random.choice([0, 1], n, p=ps)
    permutation = np.random.permutation(values)
    return (values == permutation).mean()


n = 100000
print(simulate(0.5, n))
print(simulate(0.25, n))
print(simulate(0.9, n))

**Exercise**: What is the expected accuracy of a classifier which classifies everything as the majoritary class, when the positive / negative proportions are:

* 0.1 / 0.9
* 0.4 / 0.6
* 0.75 / 0.25

$$Acc(p) = max(p, 1 - p)$$

So:

* 0.9
* 0.6
* 0.75

The accuracy score is relevant mostly when the classes are balanced. Otherwise, one should rely on other scores.

**Note**: It is always a good idea to compare the obtained score with the expected score of some trivial classifiers (typically random or majority vote).

**Exercise**: Assuming we have $N$ samples and a proportion $p$ of positive samples. What are the expected numbers of TP, TN, FP, FN for:

* a random (permutation) classifier
* a majority vote classifier

For a random classifier the expected numbers of TP, TN, FP and FN are:

| Type | Expected number |
|------|-----------------|
| TP | $N p^2$ |
| TN | $N (1 - p)^2$|
|FP | $N p (1 - p)$|
|FN | $N p (1 - p)$|

For the majority vote it is:

|  Type  | $p >0.5$ | $p < 0.5$ |
|----|----------|-----------|
| TP |   $Np$   |    $0$    |
| TN |   $0$    | $N(1 -p)$ |
| FP | $N(1-p)$ |     $0$     |
| FN |     $0$    | $Np$      |

#### Precision

The precision measures the proportion of positive samples (ground truth) among all the points predicted as positive:

$$ Prec = \frac{TP}{TP + FP} $$

**Exercise**: If $p$ is the proportion of positive samples, what is the expected precision score for a:

* random classifier
* majority vote classifier

* random classifier: $p$
* majority vote classifier: Prec = ($p$, Undefined)

#### Recall

The recall is the proportion of positive samples which were correctly classified among all positive samples:

$$ Recall = \frac{TP}{TP + FN} $$

The recall is also called the **true positive rate**

**Exercise**: If $p$ is the proportion of positive samples, what is the expected recall score for a:

* random classifier
* majority vote classifier

* random classifier: $p$
* majority vote: ($1$, $0$)

#### F1-score

Intuitively we understand that a good model should have both high precision and high recall. The F1-score corresponds to the harmonic mean of the precision and the recall:

$$ F_1 = \frac{2 \times Prec \times Recall}{Prec + Recall} $$

Compared to the arithmetic mean, the harmonic mean is biased towards the smaller values.

#### False Positive Rate

The False positive rate (a.k.a false alarm ratio) measures the proportion of negative samples which are wrongly classified as positive:

$$ FPR = \frac{FP}{FP + TN} $$

#### Sensitivity and Specificity

In biomedial fields, one often encounters the sensitivity and specificity scores:

- Sensitivity is the proportion of positive samples that are correclty predicted as such; this is another name for recall;

- Specificity is the proportion of negative samples that are correctly predicted as such; one can consider specificity as the ability to reject negatives samples.

$$ Sens = \frac{TP}{TP + FN} = TPR = Recall $$

$$ Spec = \frac{TN}{TN + FP} = TNR = 1 - FPR $$


#### Example

The figure below shows a dataset made of 2 Gaussian blobs which partially overlap:

In [ ]:
from sklearn.datasets import make_moons, make_blobs
from sklearn.metrics import classification_report

X_, y_ = make_blobs(
    n_samples=200, n_features=2, centers=2, cluster_std=7.0, random_state=1
)
_ = sns.scatterplot(x=X_.T[0], y=X_.T[1], hue=y_)

Let's train a simple model (we shall explain how it works later):

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

Xtrain_, Xtest_, ytrain_, ytest_ = train_test_split(X_, y_, random_state=1)
clf = LogisticRegression()
clf.fit(Xtrain_, ytrain_)

As with regression estimators, we can use the `predict()` method to perform predictions on the test set:

In [ ]:
ypred_ = clf.predict(Xtest_)
ypred_

Scikit-learn provides some functions to easily compute most classification scores:

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

sns.heatmap(confusion_matrix(ytest_, ypred_), annot=True)
plt.xlabel("Prediction")
plt.ylabel("Ground truth")
print(classification_report(ytest_, ypred_))

#### Decision function / probability-based measures

Most classification algorithms compute a continuous quantity which measures the degree of confidence that a sample belongs to the positive class. This quantity is often used as a score to sort test samples according to their level of certainty.

This **sample score** can be provided in two different forms:

* the **decision function** (an unbounded continuous value, with 0 used as a pivot between classes)
* the **positive class probability** which is a value between 0 and 1.

In Scikit-learn, those two quantities can be retrieved respectively using the two following methods:

* `decision_function(Xtest)` which returns a 1d array with the same length as `Xtest`
* `predict_proba(Xtest)` which returns a 2d array of shape `(len(Xtest), 2)`. The probability of the positive class being usually the second column.

Most Scikit-learn models provide at least one of those two methods.

Usually, the actual class prediction (returned by the `predict()` method) consists in considering as positive:

* the samples with a positive decision function value;
* the samples with a positive class probability above 0.5.


#### Scores

A note about the word *score*: depending on the context, score can mean different things:

- It may refer to the *sample score* predicted by the model, that we just defined;

- It can also correspond to the *attrition score* or *risk* of an employee; here, because of our design decisions, the attrition score and sample score are the same ;

- Score is also commonly used for the performance measures (like accuracy, precision, etc...) that were explained earlier.

The figure below shows the decision function of the linear model above. The black line represents the decision border. The test points are displayed with a white edge.

In [ ]:
from mll1 import plotting

In [ ]:
plt.figure(figsize=(8, 6))
sns.scatterplot(x=X_.T[0], y=X_.T[1], hue=y_, palette=["tab:red", "tab:blue"])
plt.scatter(*Xtest_.T, lw=2, edgecolor="white", c="none")
plotting.plot_decision_function(clf)
_ = plt.legend(title="class", loc="upper left")

**Note**: you can have a look at the `mll1.plotting.plot_decision_function` to see the actual implementation.

The default classification result is obtained by applying an ***a priori* threshold to the sample score**. It is thus possible to scan the range of the sample score values and see how different classification performance measures evolve.

Since in general we cannot have perfect classifiers, obviously, some performance measures will evolve in opposite directions as a function of the threshold value.

To understand how it works, let's first see how the sample score value (the output of the `decision_function` or `predict_proba` function) is distributed between positive and negative samples:

In [ ]:
from sklearn.metrics import precision_recall_curve
from matplotlib import cm

yscore_ = clf.decision_function(Xtest_)

sns.kdeplot(x=yscore_, hue=ytest_)
_ = plt.xlabel("decision function")

We can see that :

* the score distribution of the positive samples has a long tail on the left
* the distributions do not overlap for highly positive scores (> 5)

Obviously the different *classification scores* will evolve differently as we scan all the possible *sample score* thresholds:

In [ ]:
print(classification_report(ytest_, (yscore_ > 0.0).astype("int")))

In [ ]:
print(classification_report(ytest_, (yscore_ > 5.0).astype("int")))

##### Multi-criterion scores

Since we cannot have a perfect classifier, it is common to try to optimize one performance measure while keeping another one under a given threshold. The choice of the performance measures will depend on the actual use-case.

It is then interesting to build some curves where we see the evolution of a pair of criteria as we scan the sample score values. This allows us to:

* have a synthetic view of the model for all its **operating points** (i.e. a given point of the curve);
* compare several models and choose the best one for the desired operating point (since curves can cross).

##### Precision-recall curve

As its name suggests, the precision-recall curve plots the evolution of the precision and the recall when scanning the possible threshold values.

We can use the `sklearn.metrics.precision_recall_curve()` function to compute the precision and recall scores for each threshold value. Note that a point with precision 1 and recall 0 is added at the end of the corresponding arrays:

In [ ]:
from sklearn.metrics import precision_recall_curve

prec, rec, thresh = precision_recall_curve(ytest_, yscore_)

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(thresh, prec[:-1])
plt.ylabel("Precision")
plt.xlabel("Decision threshold")

plt.subplot(1, 2, 2)
plt.plot(thresh, rec[:-1])
plt.ylabel("Recall")
_ = plt.xlabel("Decision threshold")

We can even visualize the contribution of positive and negative samples:

In [ ]:
isort = np.argsort(yscore_)
mask_pos = ytest_[isort] == 1

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(thresh, prec[:-1])
plt.ylabel("Precision")
plt.xlabel("Decision threshold")
plt.scatter(thresh[mask_pos], prec[:-1][mask_pos], c="tab:blue", label="positive")
plt.scatter(thresh[~mask_pos], prec[:-1][~mask_pos], c="tab:red", label="negative")
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(thresh, rec[:-1])
plt.ylabel("Recall")
plt.xlabel("Decision threshold")
plt.scatter(thresh[mask_pos], rec[:-1][mask_pos], c="tab:blue", label="positive")
plt.scatter(thresh[~mask_pos], rec[:-1][~mask_pos], c="tab:red", label="negative")
_ = plt.legend()

Those curves are more easily read starting from the highest score values.

We can see that:

* The precision 1 starts with score values >= 1.5 (and not 5 as we could have thought looking at the decision function KDE some paragraphs above, but this is due to the smoothing effect of the KDE);
* The lowest precision is 0.5 which corresponds to the **ratio of positive samples** in the dataset;
* The recall starts from `1/n_pos_samples` on the right and ends at 1 when all the samples are classified as positive;
* Sequences of negative samples corresponds to flat values of the recall.

Note that **precision is not necessarily a monotonous function of the score value** while the recall is.

We can now visualize Precision as a function of Recall:

In [ ]:
prec, rec, thres = precision_recall_curve(ytest_, yscore_)
plt.plot(rec, prec, "-")

plt.scatter(rec[:-1][mask_pos], prec[:-1][mask_pos], c="tab:blue", label="positive")
plt.scatter(rec[:-1][~mask_pos], prec[:-1][~mask_pos], c="tab:red", label="negative")
plt.legend()
plt.xlabel("Recall")
_ = plt.ylabel("Precision")

We can note that:

* the vertical parts corresponding to sequences of consecutive negative samples;
* the fact that the curve is not monotonous.


**Exercise**: What are the expected precision-recall curves for:

* a perfect classifier
* a random classifier


* For **the perfect classifier**, all positive samples are scored higher than negative samples, so:

    * As we descrease the decision threshold from the maximum score to the minium score of the positive samples, the precision is 1 and the recall grows from 0 to 1;
    * Once we start scanning decision thresholds corresponding to negarive scores, the recall remains unchanged and and the precision drops to reach the proportion of positive samples.

* For a **random classifier**, assuming scores are uniformly drawn on an interval, we expect (on average) a constant precision corresponding to the proportion of positive samples and a recall going from 0 to 1.

* The perfect classifier: here is a detailed illustration with the following hypotheses:
    - the positive samples have a score between 0 and 5   (an arbitrary upper bound)
    - the negative samples have a score between -5 and 0  (an arbitrary lower bound)

In [ ]:
with plt.style.context("default"):
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 3, 1)
    plt.grid(axis="both", which="major", color="#ccc", linestyle="-")
    plt.plot([-5, 0, 5], [0.5, 1, 1], "-")
    plt.scatter(
        np.linspace(-5, 0, 10), np.linspace(0.5, 1, 10), c="tab:red", label="negative"
    )
    plt.scatter(
        np.linspace(0, 5, 10), np.linspace(1, 1, 10), c="tab:blue", label="positive"
    )
    plt.ylim([-0.1, 1.1])
    plt.xticks(fontweight="bold")
    plt.yticks(fontweight="bold")
    plt.xlabel("Threshold", fontweight="bold")
    plt.ylabel("Precision", fontweight="bold")
    plt.legend()

    plt.subplot(1, 3, 2)
    plt.grid(axis="both", which="major", color="#ccc", linestyle="-")
    plt.plot([-5, 0, 5], [1, 1, 0], "-")
    plt.scatter(
        np.linspace(-5, 0, 10), np.linspace(1, 1, 10), c="tab:red", label="negative"
    )
    plt.scatter(
        np.linspace(0, 5, 10), np.linspace(1, 0, 10), c="tab:blue", label="positive"
    )
    plt.ylim([-0.1, 1.1])
    plt.xticks(fontweight="bold")
    plt.yticks(fontweight="bold")
    plt.xlabel("Threshold", fontweight="bold")
    plt.ylabel("Recall", fontweight="bold")
    plt.legend()

    plt.subplot(1, 3, 3)
    plt.grid(axis="both", which="major", color="#ccc", linestyle="-")
    plt.plot([1, 1, 0], [0.5, 1, 1], "-")
    plt.scatter(
        np.linspace(1, 1, 10), np.linspace(1, 0.5, 10), c="tab:red", label="negative"
    )
    plt.scatter(
        np.linspace(0, 1, 10), np.linspace(1, 1, 10), c="tab:blue", label="positive"
    )
    plt.xlim([-0.1, 1.1])
    plt.ylim([-0.1, 1.1])
    plt.xlabel("Recall", fontweight="bold")
    plt.ylabel("Precision", fontweight="bold")
    plt.xticks(fontweight="bold")
    plt.yticks(fontweight="bold")
    plt.legend()
    plt.suptitle("Perfect classifier", fontweight="bold")
    plt.tight_layout()

On the first figure (precision):
- we can see that all negative (red) samples are on the left side, below the 0 threshold;
- so if we use a threshold smaller than 0, we will predict more and more negative samples as positives, to the precision will be between 0.5 (the total proportion of positives) and 1 (only positive samples are selected, hence "perfect" classifier)
- if the threshold is increased above 0, the proportion of true positives will not change since only blues (positives) are selected

On the second figure (recall):
- for thresholds below 0, all samples are predicted as positive, so all actual positives are kept. Hence a recall of 1.
- for thresholds > 0, we mispredict more and more positives as negatives, ending in 0 recall.

Finally the precision/recall figure:
- from left to right, recall increases from 0 to 1 while the threshold (not shown) decreases from 5 to 0.
- when we reach recall=1 (threshold=0), we start scanning the negative samples, recall stays constant while the precision vertically drops until it reaches 0.5, the ratio of positive samples in this example.

* For a random classifier: the following figure makes the hypothesis that the score is distributed uniformly between -5 and 5 (no matter if the sample is positive or negative):

In [ ]:
with plt.style.context("default"):
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 3, 1)
    plt.grid(axis="both", which="major", color="#ccc", linestyle="-")
    plt.plot([-5, 0, 5], [0.5, 0.5, 0.5], "-")
    plt.ylim([-0.1, 1.1])
    plt.xticks(fontweight="bold")
    plt.yticks(fontweight="bold")
    plt.xlabel("Threshold", fontweight="bold")
    plt.ylabel("Precision", fontweight="bold")

    plt.subplot(1, 3, 2)
    plt.grid(axis="both", which="major", color="#ccc", linestyle="-")
    plt.plot([-5, 5], [1, 0], "-")
    plt.ylim([-0.1, 1.1])
    plt.xticks(fontweight="bold")
    plt.yticks(fontweight="bold")
    plt.xlabel("Threshold", fontweight="bold")
    plt.ylabel("Recall", fontweight="bold")

    plt.subplot(1, 3, 3)
    plt.grid(axis="both", which="major", color="#ccc", linestyle="-")
    plt.plot([0, 1], [0.5, 0.5], "-")
    plt.xlim([-0.1, 1.1])
    plt.ylim([-0.1, 1.1])
    plt.xticks(fontweight="bold")
    plt.yticks(fontweight="bold")
    plt.xlabel("Recall", fontweight="bold")
    plt.ylabel("Precision", fontweight="bold")
    plt.suptitle("Random classifier (average)", fontweight="bold")
    plt.tight_layout()

What is shown here should be understood in terms of *expectation* (if we average a large number of such random classifiers).

- The precision would be *on average* constant with a value corresponding to the ratio of positive samples in the dataset;
- The recall would decrease from 1 (all samples predicted as positives) to 0 (all negatives).

Let's check empirically with actual classifiers:

In [ ]:
prec, rec, thres = precision_recall_curve(ytest_, yscore_)
plt.plot(rec, prec, label="prediction")

# let all negative samples have score < 0.5, and all positive have score > 0.5
perfect_score = np.where(
    ytest_ == 0,
    0.5 * np.random.rand(len(ytest_)),
    0.5 + 0.5 * np.random.rand(len(ytest_)),
)

prec, rec, thres = precision_recall_curve(ytest_, perfect_score)
plt.plot(rec, prec, label="perfect")
print(len(prec), len(ytest_))
np.random.seed(1)
prec, rec, thres = precision_recall_curve(ytest_, np.random.rand(len(ytest_)))
plt.plot(rec, prec, label="random")
plt.plot([0, 1], [np.mean(ytest_), np.mean(ytest_)], "-", label="expected random")


plt.xlabel("Recall")
plt.ylabel("Precision")
_ = plt.legend()

Note:

* The (0, 1) additional point
* The missing decreasing part of the perfect curve (since it is useless to compute the average precision)

##### ROC curve

The ROC (Receiver Operating Characteristic) curve show the evolution of the **true positive rate** (a.k.a the recall) as a function of the **false positive rate** (ratio of negative samples classified as positive among all negative samples).

The *weird* name comes from the origin of the ROC curve during WW2 when people were analyzing the separation between signal and noise in radars.

**Exercise**:
1. What is the value of TPR and FPR when all points are classified as positive ?
2. What is the value of TPR and FPR when all points are classified as negative ?
3. Are TPR and FPR monotonous functions of the threshold value ?
4. What does the ROC curve looks like for a perfect classifier ?
5. What does the ROC curve looks like for a random classifier ?

1. TPR=1, FPR=1
2. TPR=0, FPR=0
3. Both are non-decreasing functions
4. The perfect ROC curve is `[(0, 0), (0, 1), (1, 1)]`.
5. The ROC curve of a random classifier is the [(0, 0), (1, 1)] line.

Note that depending on the actual problem at hand, there can be some variations on quantities drawn on the ROC curve. For a detection problem for instance, the FPR can be replaced by the non-normalized number of false positives (false alarms).

Let's visualize different ROC curve cases:

In [ ]:
from sklearn.metrics import roc_curve

fpr, tpr, thres = roc_curve(ytest_, yscore_)
plt.plot(fpr, tpr, label="prediction")

fpr, tpr, thres = roc_curve(ytest_, ytest_)
plt.plot(fpr, tpr, label="perfect")

np.random.seed(1)
fpr, tpr, thres = roc_curve(ytest_, np.random.rand(len(ytest_)))
plt.plot(fpr, tpr, label="random")

plt.plot([0, 1], [0, 1], label="expected random")
plt.ylabel("True Positive Rate")
plt.xlabel("False Positive Rate")
_ = plt.legend()

##### Area under curve

So the curves shown above allow us to compare several models at some specific operating points.

For both ROC and precision-recall curves, a model can be better for all operating points if its curve is always above the other curves.

We have seen that in both cases, the curve of a perfect model corresponds to a curve with a unit area (y values is 1 over all the [0, 1] x axis except for a discrete set of points (1 point actually).

So intuitively we see that the area under the curve can aggregate the overall performance of the model.

**Average precision**: is computed as the weighted average of the precision values where the weight corresponds to the recall increment between the current point and the previous one. It corresponds the the area under the curve where the precision value is not interpolated but constant between consecutive points.

**Area under the ROC curve** or ROC AUC is more obvious to compute since it is non-decreasing.

Fortunately, Scikit-learn provides some functions to quickly perform those analyses:

In [ ]:
from sklearn import metrics

avg_prec = metrics.average_precision_score(ytest_, yscore_)
print(f"Avg Prec: {avg_prec:0.3f}")

precision, recall, thresholds = metrics.precision_recall_curve(ytest_, yscore_)
avg_disp = metrics.PrecisionRecallDisplay(
    precision=precision, recall=recall, average_precision=avg_prec
)
_ = avg_disp.plot()

In [ ]:
roc_auc = metrics.roc_auc_score(ytest_, yscore_)
print(f"ROC AUC: {roc_auc:0.3f}")

fpr, tpr, thres = roc_curve(ytest_, yscore_)

_ = metrics.RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc).plot()

## Classification algorithms

Let's come back at our attrition dataset.

In [ ]:
df_attr

As preparation for performing a classification task, we will use the `has_left` variable as the **target**, and all other variables except the `Emp ID` as the **features**.

Then we partition the dataset into a **training** and a **test** parts.

In [ ]:
from sklearn.model_selection import train_test_split

X = df_attr.drop(["Emp ID", "has_left"], axis=1)
y = df_attr.has_left.values

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, random_state=1)

### Linear Models

Binary classification linear models are based on a linear decision function:

$$f(\boldsymbol x) = \boldsymbol w^T \boldsymbol x + b$$

The decision is then typically taken using the step function:

$$h(x) = \left\{
\matrix{
    1 & \text{if }x > 0 \\
    0 & \text{otherwise}
}
\right.
$$

So the prediction is:

$$ \tilde y_i = h(f(\boldsymbol x_i)) $$

Note that the intercept plays the role of a threshold $b$.

#### Logistic Regression

Logistic regression models the probability that a data point belong to class $1$ as:

$$p(y=1| \boldsymbol x, \boldsymbol w, b) = \sigma(\boldsymbol w^T \boldsymbol x + b)$$

Where $\sigma(.)$ is the sigmoid function:

$$\sigma(x) = \frac{1}{1 + e^{-x}}$$

In [ ]:
x = np.linspace(-5, 5, 101)
plt.grid("on")
plt.plot(x, (1 + np.exp(-x)) ** -1)
plt.xlabel("$x$")
plt.yticks(np.linspace(0, 1, 5))
_ = plt.ylabel("$\sigma(x)$")

This function is well suited for playing with probabilities since:

$$\sigma(-x) = 1 - \sigma(x)$$

It can also be understood as a **smooth approximation of the step function**.

Assuming the two classes are encoded as $\{-1, 1\}$, the probability that a sample is negative is then:

$$p(y=-1| \boldsymbol x, \boldsymbol w, b) = \sigma(-(\boldsymbol w^T \boldsymbol x + b))$$

So we can sum up both cases as:

$$p(y| \boldsymbol x, \boldsymbol w, b) = \sigma(y(\boldsymbol w^T \boldsymbol x + b))$$

In [ ]:
x = np.linspace(-5, 5, 101)
plt.grid("on")
plt.plot(x, (1 + np.exp(-x)) ** -1, label="y=1")
plt.plot(x, (1 + np.exp(x)) ** -1, label="y=-1")
plt.xlabel("$w^Tx + b$")
plt.yticks(np.linspace(0, 1, 5))
plt.ylabel("$p(y|x, w, b)$")
_ = plt.legend()

The likelihood that points belong to their associated classes can be computed as:

$$ p(\boldsymbol y | X, \boldsymbol w, \boldsymbol b) = \prod_{i=1}^N p(y_i| \boldsymbol x_i, \boldsymbol w, b)$$

where $\boldsymbol y = (y_1, y_2, \dots, y_N)^T$ and $\mathrm{row}(X, i) = \boldsymbol x_i^T$.

Logistic regression consists in finding the parameters $\boldsymbol w$ and $b$ maximizing the likelihood of the data points. Or equivalently it minimizes the sum of the negative log-likelihood of the data points.

$$-\log(\sigma(x)) = \log(1 + e^{-x})$$

With a bit of algrebra and assuming the target value is encoded as $\{1, -1\}$ the individual negative log likelihood for the data point $\boldsymbol x_i$ with target $y_i$ is:

$$\ell(\boldsymbol x_i, y_i; \boldsymbol w, b) = \log(1 + \exp(-y_i (\boldsymbol w^T\boldsymbol x_i + b))$$

So the individual loss function for positive and negative data points look like:

In [ ]:
x = np.linspace(-5, 5, 101)
plt.grid("on")
plt.plot(x, np.log(1 + np.exp(-x)), label="y=1")
plt.plot(x, np.log(1 + np.exp(x)), label="y=-1")
plt.xlabel("$w^T x  + b$")
plt.ylabel(r"$\ell(x, y; w, b)$")
_ = plt.legend()

Note that the function tends to zero when the decision function is far on the *correct* side, while it becomes approximatively linear on the *wrong* side.

As with least-squares regression, a regularization term can be added to avoid over-fitting as well stabilize the numerical result. The optimization problem is then:

$$\min_{\boldsymbol w, b} \mathcal R(\boldsymbol w) + C \sum_{i=1}^N \ell(\boldsymbol x_i, y_i; \boldsymbol w, b) $$

Where $\mathcal R(.)$ is a regularization term. Typically:

* $L_2$ norm: $\frac{1}{2}\|\boldsymbol w\|^2_2$
* $L_1$ norm: $\|\boldsymbol w\|_1$

Note the similarity of the regularization with least-squares regression.

In [ ]:
from mll1.plotting import plot_decision_function
from sklearn.datasets import make_blobs

X_, y_ = make_blobs(100, 2, centers=2, cluster_std=7.0, random_state=1)

clf = LogisticRegression().fit(X_, y_)

plt.figure(figsize=(6, 6))
sns.scatterplot(x=X_.T[0], y=X_.T[1], hue=y_, palette=["tab:red", "tab:blue"])
plot_decision_function(clf)

Now let's apply this to our dataset. We will :

* Use target encoding for categorical variables
* Standardize the inputs before using the classifier

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from category_encoders import TargetEncoder

clf = LogisticRegression(C=1.0, max_iter=1000)

model = Pipeline(
    [
        ("cat_enc", TargetEncoder(cols=["dept", "salary"])),
        ("scaler", StandardScaler()),
        ("clf", clf),
    ]
)

model.fit(Xtrain, ytrain)

In [ ]:
yscore = model.predict_proba(Xtest).T[1]
print(
    f"Avg Prec: {metrics.average_precision_score(ytest, yscore):0.3f} (pos. ratio: {ytest.mean(): 0.3f})"
)
print(f"ROC AUC: {metrics.roc_auc_score(ytest, yscore):0.3f}")

Note the big difference between the two scores.

To easily visualize the results :

In [ ]:
from mll1 import plotting

plotting.plot_classification_result(model, Xtest, ytest)

* The (0.0, 1.0) point is systematically added to the actual precision / recall computation so that the graph always starts at the same point.

* We see that the precision is very low on the left side, and then increases to reach a maximum of 0.6 around a recall of 0.15

This means that in our case, the samples with the higher scores are misclassified.

In [ ]:
sns.kdeplot(x=yscore, hue=ytest)
plt.xlabel("Positive probability")

#### Linear models with non-linearities

In most real-word problems, the dataset is not linearly separable. The figure below shows a toy example of a non-linearly separable dataset:

In [ ]:
from sklearn.datasets import make_moons

X_, y_ = make_moons(150, noise=0.2, random_state=1)
_ = sns.scatterplot(x=X_.T[0], y=X_.T[1], hue=y_, palette=["tab:red", "tab:blue"])

Clearly no straight line allows to separate positive and negative samples.

As for least-squares regression it is possible to apply a **polynomial expansion**.

**Exercise**: What is the minimal degree adapted to the dataset above ?

At least 3 since we need a direction change

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler

clf = LogisticRegression(C=1.0)

model = Pipeline([("poly", PolynomialFeatures(5)), ("clf", clf)]).fit(X_, y_)

plt.figure(figsize=(8, 4))
sns.scatterplot(x=X_.T[0], y=X_.T[1], hue=y_, palette=["tab:red", "tab:blue"])
plot_decision_function(model)

**Exercise**: Play with the C parameter and explain the result

High values of the C parameter force the decision boundary to follow the dataset.

Let's apply this model on our attrition dataset:

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler

clf = LogisticRegression(C=0.1, max_iter=1000)

model = Pipeline(
    [
        ("cat_enc", TargetEncoder(cols=["dept", "salary"])),
        ("scaler", StandardScaler()),
        ("poly", PolynomialFeatures(4)),
        ("clf", clf),
    ]
).fit(Xtrain, ytrain)

plotting.plot_classification_result(model, Xtest, ytest)

We can use the same tools we used previously to analyze the feature importances. In linear models, the weights can be used as a measure of the feature importance. However, in our case, because of polynomial expansion, it is difficult to retrieve the importance of original features. We can still use the permutation based estimation:

In [ ]:
from sklearn.inspection import permutation_importance
from sklearn.metrics import make_scorer, average_precision_score

avg_prec_scorer = make_scorer(
    average_precision_score, greater_is_better=True, needs_proba=True
)

result = permutation_importance(model, Xtest, ytest, scoring=avg_prec_scorer)
_ = (
    pd.DataFrame({"importance": result["importances_mean"], "feature": Xtest.columns})
    .set_index("feature")
    .sort_values("importance")
    .plot.barh()
)

Note how we defined a `scorer` object to be used with the `permutation_importance` function. The same kind of object can be used to customize the score used during hyper-parameter optimization for instance.

Most score functions have the following signature `score_func(ytrue, ypred)` with `ypred` being either the actual prediction or the score.

Scorer objects have the following signature: `scorer(model, Xtest, ytest)`. The `make_scorer()` function allows to transform a score function into a scorer object. The parameters allow to indicate which type of scoring function is used.

#### Support Vector Classification

The idea of the support vector algorithm is to penalize only the points which are on the "wrong side" of the decision function (null contribution of points on the "good" side). To enforce a good separation, a "security margin" is added.

The null part of the loss functions allows for powerful optimization in the solving algorithm. Depending on the implementation $L_1$ and $L_2$ regularizations are possible.

We illustrate below the two loss functions available with the `sklearn.svm.LinearSVC` estimator:

In [ ]:
x = np.linspace(-5, 5, 101)

plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.title("hinge loss")
plt.plot(x, np.maximum(0, -x + 1), label="positive")
plt.plot(x, np.maximum(0, x + 1), label="negative")

plt.subplot(1, 2, 2)
plt.title("squared hinge loss")
plt.plot(x, np.maximum(0, -x + 1) ** 2, label="positive")
_ = plt.plot(x, np.maximum(0, x + 1) ** 2, label="negative")

Compared to logistic regression, the SVC algorithm can be **faster to train**, thanks to the null part of the loss function. However there is **no direct interpretation in terms of probabilities**.

The hinge loss is usually more robust to outliers (because of the linear growth of the penalty).

**Note*:** The "support vector" name comes from the fact that, due to the null parts of the loss function, only the points on the wrong side of the margin have a contribution in the final solution and thus "support" the decision margin.

In [ ]:
from sklearn.svm import LinearSVC

X_, y_ = make_blobs(100, 2, centers=2, cluster_std=6.0, random_state=1)

clf = LinearSVC(C=100.0, random_state=1).fit(X_, y_)

plt.figure(figsize=(6, 6))
sns.scatterplot(x=X_.T[0], y=X_.T[1], hue=y_, palette=["tab:red", "tab:blue"])
plot_decision_function(clf)

In [ ]:
from sklearn.svm import LinearSVC

clf = LinearSVC(C=0.1, loss="squared_hinge", random_state=1)

model = Pipeline(
    [
        ("cat_enc", TargetEncoder(cols=["dept", "salary"])),
        ("std", StandardScaler()),
        ("poly", PolynomialFeatures(5)),
        ("clf", clf),
    ]
).fit(Xtrain, ytrain)

plotting.plot_classification_result(model, Xtest, ytest)

**Exercise**: Play with the parameters:

* `C`: fitness / regularization trade off
* `loss`: `hinge` or `squared hinge`
* `penalty`: `l1` or `l2`

**Note** that not all combinations of loss functions and penalty / regularizers are possible. They depend on the available solvers. For instance, only `squared_hinge` can be used with `l1` regularization (and `dual=False`) due to the solver used.

### Nearest Neighbors Classification

The nearest neighbors classification is again very easy to understand. To classify a new point, we examine its $K$ nearest neighbors in the input space and compute the proportion of positive / negative samples among those neighbors. This proportion is thus returned by the `predict_proba()` method.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import make_moons

X_, y_ = make_moons(150, noise=0.2, random_state=1)

clf = KNeighborsClassifier()

model = Pipeline([("std", StandardScaler()), ("clf", clf)]).fit(X_, y_)

plt.figure(figsize=(8, 4))
sns.scatterplot(x=X_.T[0], y=X_.T[1], hue=y_, palette=["tab:red", "tab:blue"])
plot_decision_function(model)

As already noted when we talked about regression, an implicit assumption is that the distance in the input space has a consistent meaning. In the absence of further information, once should at least rescale the input features so that they have similar scales (standardize, min-max scaling, etc.)

Applying it to our case, it gives:

In [ ]:
clf = KNeighborsClassifier(n_neighbors=5)

model = Pipeline(
    [
        ("cat_enc", TargetEncoder(cols=["dept", "salary"])),
        ("std", StandardScaler()),
        ("clf", clf),
    ]
).fit(Xtrain, ytrain)

plotting.plot_classification_result(model, Xtest, ytest)

### Decision-Tree based algorithm

The decision tree algorithm seen in the regression case is easily adaptable to the classification case. Indeed the  main difference will rely in the criterion which is optimized at each node split. While in regression, we would typically try to minimize the mean squared error, in classification the possible criteria are typically:

* the Gini criterion: $\sum_{k=1}^{K} p_{mk}(1 - p_{mk})$
* the entropy: $\sum_{k=1}^K -p_{mk}\log(p_{mk})$

Where $p_{mk}$ is the proportion of class $k$ for the terminal (a.k.a leaf) node $m$.

In [ ]:
from scipy.special import xlogy

x = np.linspace(0, 1, 101)
plt.plot(x, 2 * x * (1 - x), label="Gini")
plt.plot(x, -xlogy(x, x) - xlogy((1 - x), (1 - x)), label="Entropy")
plt.xlabel("$p_{m1}$")
plt.ylabel("Criterion value")
_ = plt.legend()

Let's illustrate the way it works on the double moons toy dataset:

In [ ]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(min_samples_leaf=10)
clf.fit(X_, y_)
plt.figure(figsize=(8, 4))
sns.scatterplot(x=X_.T[0], y=X_.T[1], hue=y_, palette=["tab:red", "tab:blue"])
plot_decision_function(clf)

It is of course possible to build random forests of decision tree classifiers:

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=100, min_samples_leaf=10)
clf.fit(X_, y_)
plt.figure(figsize=(8, 4))
sns.scatterplot(x=X_.T[0], y=X_.T[1], hue=y_, palette=["tab:red", "tab:blue"])
plot_decision_function(clf)

And to use any of the gradient boosting variants:

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

clf = GradientBoostingClassifier(n_estimators=100, min_samples_leaf=10)
clf.fit(X_, y_)
plt.figure(figsize=(8, 4))
sns.scatterplot(x=X_.T[0], y=X_.T[1], hue=y_, palette=["tab:red", "tab:blue"])
plot_decision_function(clf)

## Next notebook

Model and hyper-parameters selection, validation strategies and more...